# Scraping Steam reviews of Baldur's Gate 3

In [2]:
# Import necessary packages
import requests
import pandas as pd
import time
from config import BG3_data_file_path

In [3]:
# FUnction to fetch reviews
def fetch_steam_reviews(appid, num_reviews=500):
    reviews = []
    cursor = '*'
    language = 'english'
    
    while len(reviews) < num_reviews:
        url = f"https://store.steampowered.com/appreviews/{appid}?json=1&filter=all&language={language}&num_per_page=100&cursor={cursor}&day_range=365"
        response = requests.get(url)
        
        if response.status_code != 200:
            print(f"Failed to get reviews: {response.status_code}")
            break
        
        data = response.json()
        
        if data['success'] == 1:
            reviews += data['reviews']
            cursor = data['cursor']
        else:
            print("Failed to get reviews: API returned unsuccessful status")
            break
        
        if len(reviews) >= num_reviews:
            reviews = reviews[:num_reviews]
        
        # Sleep to avoid Steam rate-limiting
        time.sleep(1)
    
    return reviews

appid = '1086940' # Bauldur's Gate 3
reviews = fetch_steam_reviews(appid)
print(f"Fetched {len(reviews)} reviews.") # Message printed if function successful

Failed to get reviews: API returned unsuccessful status
Fetched 400 reviews.


In [22]:
# Create pandas dataframe with reviews
review_df = pd.DataFrame(reviews)
# Rename review column
review_df = review_df.rename(columns={'review_text':'review'})
# Export dataframe to csv file 
data_file_path = BG3_data_file_path
review_df.to_csv(data_file_path, index=False)
review_df.head()

,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location
0,146197289,"{'steamid': '76561198209057388', 'num_games_ow...",english,my work is really cutting into my Baldur's Gat...,1694565640,1694565640,True,7830,2987,0.980962038040161133,0,True,False,False,True,
1,153859247,"{'steamid': '76561198412806684', 'num_games_ow...",english,You can convince bosses to kill themselves. 10/10,1702909919,1702909919,True,3605,900,0.98073875904083252,10,True,False,False,True,
2,143924257,"{'steamid': '76561198022720435', 'num_games_ow...",english,Possessed a guard and had them open a gate con...,1691783669,1691783669,True,3124,1268,0.978952229022979736,12,True,False,False,True,
3,146776206,"{'steamid': '76561198007607100', 'num_games_ow...",english,I have only one point of criticism:\nThis game...,1695386806,1701885967,True,1320,6,0.97847980260848999,28,True,False,False,True,
4,143578821,"{'steamid': '76561198004904749', 'num_games_ow...",english,No microtransactions\nNo in-game purchases\nNo...,1691401740,1691401740,True,15452,51,0.978366076946258545,0,True,False,False,True,


In [12]:
# View info about dataframe
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   recommendationid             500 non-null    object
 1   author                       500 non-null    object
 2   language                     500 non-null    object
 3   review                       500 non-null    object
 4   timestamp_created            500 non-null    int64 
 5   timestamp_updated            500 non-null    int64 
 6   voted_up                     500 non-null    bool  
 7   votes_up                     500 non-null    int64 
 8   votes_funny                  500 non-null    int64 
 9   weighted_vote_score          500 non-null    object
 10  comment_count                500 non-null    int64 
 11  steam_purchase               500 non-null    bool  
 12  received_for_free            500 non-null    bool  
 13  written_during_early_access  500 no

In [14]:
# Show summary stats for dataframe
review_df.describe()

,timestamp_created,timestamp_updated,votes_up,votes_funny,comment_count
count,5.000000e+02,5.000000e+02,500.000000,500.000000,500.000000
mean,1.697293e+09,1.698477e+09,650.426000,104.406000,5.418000
std,8.068341e+06,8.393158e+06,1692.239412,319.628663,21.035976
min,1.689325e+09,1.689353e+09,5.000000,0.000000,0.000000
25%,1.691524e+09,1.691719e+09,9.000000,0.000000,0.000000
50%,1.692991e+09,1.694564e+09,41.000000,3.000000,0.000000
75%,1.701094e+09,1.702781e+09,478.250000,36.500000,1.000000
max,1.720615e+09,1.720636e+09,15814.000000,2987.000000,255.000000


In [34]:
# Check whether there are duplicates
duplicates = review_df.duplicated(subset='review', keep='first')
duplicate_entries = review_df[duplicates]
# Show how many duplicates there are. Too many might be a sign of an issue with the fetching process (e.g., some Steam hiccup)
print(len(duplicate_entries))

5
